In [27]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from tqdm import tqdm
from cw import L2Adversary
from torchvision.utils import save_image
import tensorflow as tf
from numpy import moveaxis

In [28]:
# Test Dataset
transformation = transforms.Compose([transforms.ToTensor()])
test_directory = "test_fake"
test_imgfolder = datasets.ImageFolder(test_directory, transform=transformation)
test_dataloader = torch.utils.data.DataLoader(test_imgfolder, batch_size=1, shuffle=True)

In [29]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [30]:
model = torch.load("Models/resnet_base.pt")
model.cuda()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [31]:
adversary = L2Adversary(targeted=False,
                           confidence=1.0,
                           c_range=(1e-7, 1e-4),
                           search_steps=1,
                           max_steps=1000,
                           optimizer_lr=1e-2,
                           init_rand=True)

In [32]:
def test(examples, model, device):
    num_correct = 0
    for img in examples:
        img = img.to(device)
        if(model(img).squeeze() < 0):
            num_correct += 1
    return num_correct/len(examples)

In [33]:
examples = []
count = 0
for img,label in tqdm(test_dataloader):
    img = img.to(device)
    label = label.to(device)
    res = adversary(model, img, label, device)
    examples.append(res)
    
    count += 1
    if count > 100:
        break





  0%|          | 0/4999 [00:00<?, ?it/s]

Using scale consts: [1e-07]
batch [0] loss: 0.056437134742736816
batch [10] loss: 1.0690791606903076
batch [20] loss: 0.3464754521846771
batch [30] loss: 0.05541723221540451
batch [40] loss: 0.01816614717245102
batch [50] loss: 0.008865579962730408
batch [60] loss: 0.004074404947459698
batch [70] loss: 0.002162997145205736
batch [80] loss: 0.0015084769111126661
batch [90] loss: 0.001219652476720512
batch [100] loss: 0.0010670882184058428
batch [110] loss: 0.0010232070926576853
batch [120] loss: 0.0010094671742990613
batch [130] loss: 0.0010028383694589138
batch [140] loss: 0.0010010713012889028
batch [150] loss: 0.0010003006318584085
batch [160] loss: 0.0010000255424529314
batch [170] loss: 0.0009999442845582962
batch [180] loss: 0.0009999104076996446
batch [190] loss: 0.0009998977184295654
batch [200] loss: 0.0009998935274779797
batch [210] loss: 0.0009998921304941177
batch [220] loss: 0.0009998916648328304
batch [230] loss: 0.0009998914320021868
batch [240] loss: 0.000999891432002186





  0%|          | 1/4999 [00:02<3:59:24,  2.87s/it]

batch [280] loss: 0.0009998914320021868
batch [290] loss: 0.0009998914320021868
batch [300] loss: 0.0009998914320021868
Using scale consts: [1e-07]
batch [0] loss: 0.024394117295742035
batch [10] loss: 0.4538916349411011
batch [20] loss: 0.1473754495382309
batch [30] loss: 0.024056240916252136
batch [40] loss: 0.008281963877379894
batch [50] loss: 0.0043404074385762215
batch [60] loss: 0.0023106620647013187
batch [70] loss: 0.0014947461895644665
batch [80] loss: 0.0012147794477641582
batch [90] loss: 0.0010922695510089397
batch [100] loss: 0.00102781702298671
batch [110] loss: 0.0010092377196997404
batch [120] loss: 0.0010033625876531005
batch [130] loss: 0.0010005758376792073
batch [140] loss: 0.0009998148307204247
batch [150] loss: 0.0009994924766942859
batch [160] loss: 0.0009993761777877808
batch [170] loss: 0.0009993409039452672
batch [180] loss: 0.0009993265848606825
batch [190] loss: 0.0009993212297558784
batch [200] loss: 0.000999319483526051
batch [210] loss: 0.000999318901449





  0%|          | 2/4999 [00:05<3:58:22,  2.86s/it]

...

batch [300] loss: 0.0009992375271394849
Using scale consts: [1e-07]
batch [0] loss: 0.058257732540369034
batch [10] loss: 1.106374979019165
batch [20] loss: 0.3581748604774475
batch [30] loss: 0.05707552284002304
batch [40] loss: 0.018700774759054184
batch [50] loss: 0.00913020595908165
batch [60] loss: 0.004184188321232796
batch [70] loss: 0.002204437740147114
batch [80] loss: 0.0015235210303217173
batch [90] loss: 0.0012271814048290253
batch [100] loss: 0.001068963436409831
batch [110] loss: 0.0010235473746433854
batch [120] loss: 0.0010093682212755084
batch [130] loss: 0.0010024959919974208
batch [140] loss: 0.0010006692027673125
batch [150] loss: 0.0009998729219660163
batch [160] loss: 0.000999587937258184
batch [170] loss: 0.0009995041182264686
batch [180] loss: 0.0009994690772145987
batch [190] loss: 0.0009994559222832322
batch [200] loss: 0.0009994514985010028
batch [210] loss: 0.0009994501015171409
batch [220] loss: 0.0009994496358558536
batch [230] loss: 0.00099944940302521
ba





  2%|▏         | 99/4999 [04:45<4:05:11,  3.00s/it]

batch [290] loss: 0.000999449286609888
batch [300] loss: 0.000999449286609888
Using scale consts: [1e-07]
batch [0] loss: 0.0369587317109108
batch [10] loss: 0.6906229853630066
batch [20] loss: 0.2239132970571518
batch [30] loss: 0.03613213449716568
batch [40] loss: 0.012100405059754848
batch [50] loss: 0.006078434642404318
batch [60] loss: 0.002984705613926053
batch [70] loss: 0.0017514752689749002
batch [80] loss: 0.0013281055726110935
batch [90] loss: 0.0011412237072363496
batch [100] loss: 0.0010427454253658652
batch [110] loss: 0.0010145045816898346
batch [120] loss: 0.001005573314614594
batch [130] loss: 0.0010013136779889464
batch [140] loss: 0.0010001647751778364
batch [150] loss: 0.0009996698936447501
batch [160] loss: 0.0009994924766942859
batch [170] loss: 0.0009994395077228546
batch [180] loss: 0.0009994176216423512
batch [190] loss: 0.0009994094725698233
batch [200] loss: 0.0009994067950174212
batch [210] loss: 0.0009994058636948466
batch [220] loss: 0.0009994055144488811






  2%|▏         | 100/4999 [04:48<4:04:08,  2.99s/it]

batch [280] loss: 0.0009994053980335593
batch [290] loss: 0.0009994053980335593
batch [300] loss: 0.0009994053980335593
Using scale consts: [1e-07]
batch [0] loss: 0.037058643996715546
batch [10] loss: 0.6909849047660828
batch [20] loss: 0.224196657538414
batch [30] loss: 0.036339301615953445
batch [40] loss: 0.012162929400801659
batch [50] loss: 0.006104324944317341
batch [60] loss: 0.002999147167429328
batch [70] loss: 0.0017596883699297905
batch [80] loss: 0.0013304364401847124
batch [90] loss: 0.0011420956579968333
batch [100] loss: 0.0010440152836963534
batch [110] loss: 0.0010159250814467669
batch [120] loss: 0.0010068443370983005
batch [130] loss: 0.0010026386007666588
batch [140] loss: 0.0010014650179073215
batch [150] loss: 0.0010009751422330737
batch [160] loss: 0.001000798074528575
batch [170] loss: 0.0010007434757426381
batch [180] loss: 0.0010007217060774565
batch [190] loss: 0.0010007136734202504
batch [200] loss: 0.0010007109958678484
batch [210] loss: 0.0010007100645452

In [36]:
test(examples, model, device)

0.9801980198019802

In [37]:
model2 = torch.load("Models/resnet_huber_regularized.pt")
model2.cuda()
test(examples, model2, device)

0.9702970297029703